In [1]:
# Creating Date List For Backtesting
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import backtesting.utilities as bu
dates_output = bu.get_backtesting_dates(date_to=20160226,years_back=10)
double_dates = sorted(dates_output['double_dates'],reverse=False)
selected_dates = double_dates
selected_dates


[20060301,
 20060308,
 20060315,
 20060322,
 20060329,
 20060405,
 20060412,
 20060419,
 20060426,
 20060503,
 20060510,
 20060517,
 20060524,
 20060531,
 20060607,
 20060614,
 20060621,
 20060628,
 20060705,
 20060712,
 20060719,
 20060726,
 20060802,
 20060809,
 20060816,
 20060823,
 20060830,
 20060906,
 20060913,
 20060920,
 20060927,
 20061004,
 20061011,
 20061018,
 20061025,
 20061101,
 20061108,
 20061115,
 20061122,
 20061129,
 20061206,
 20061213,
 20061220,
 20061227,
 20070103,
 20070110,
 20070117,
 20070124,
 20070131,
 20070207,
 20070214,
 20070221,
 20070228,
 20070307,
 20070314,
 20070321,
 20070328,
 20070404,
 20070411,
 20070418,
 20070425,
 20070502,
 20070509,
 20070516,
 20070523,
 20070530,
 20070606,
 20070613,
 20070620,
 20070627,
 20070711,
 20070718,
 20070725,
 20070801,
 20070808,
 20070815,
 20070822,
 20070829,
 20070905,
 20070912,
 20070919,
 20070926,
 20071003,
 20071010,
 20071017,
 20071024,
 20071031,
 20071107,
 20071114,
 20071121,
 20071128,

In [3]:
# Historical Butterfly Report Run:

import opportunity_constructs.futures_butterfly as fb

for date_to in selected_dates:
    fb.generate_futures_butterfly_sheet_4date(date_to=date_to)
    

    

In [2]:
import backtesting.backtest_futures_with_paths as bfp
butterfly_output = bfp.get_backtest_summary(date_list = selected_dates,
                                            use_existing_filesQ=True)

big_data = butterfly_output['big_data']

# re check the restricted buy signal
# time series properties per tickerhead
# unproportionate contract weights




In [5]:
import backtesting.backtest_futures_butterfly as bfb
total_pnl_frame_3 = bfb.construct_futures_butterfly_portfolio(rule_no=3,
                                          backtest_output=backtest_output,
                                          date_list = selected_dates,
                                          pnl_field='hold_pnl5')

total_pnl_frame_1 = bfb.construct_futures_butterfly_portfolio(rule_no=1,
                                          backtest_output=backtest_output,
                                          date_list = selected_dates,
                                          pnl_field='hold_pnl5')

In [16]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(total_pnl_frame_1.index,total_pnl_frame_1['portfolio'].cumsum(),
         total_pnl_frame_3.index,total_pnl_frame_3['portfolio'].cumsum())
plt.grid()
plt.legend([1,3])
plt.show()


In [13]:
import math as m
sharp_1 = m.sqrt(52)*total_pnl_frame_1['portfolio'].mean()/total_pnl_frame_1['portfolio'].std()
sharp_3 = m.sqrt(52)*total_pnl_frame_3['portfolio'].mean()/total_pnl_frame_3['portfolio'].std()

print(sharp_1)
print(sharp_3)

4.534496285051776
4.896041277820497


In [7]:
import backtesting.backtest_futures_butterfly as bfb
portfolio_contributors_output = bfb.analyze_portfolio_contributors(date_list=selected_dates,
                                   backtest_output=backtest_output,
                                  rule_no=1,
                                  date_from=20160127,
                                  date_to=20160224)

portfolio_contributors_output['ticker_head_total_pnls']

portfolio_contributors_output['total_pnl_frame']

portfolio_contributors_output['ticker_head_total_pnls']

,abs_pnl,pnl,ticker_head
CL,12563,12563,CL
HO,10190,10190,HO
NG,8833,-8833,NG
B,8711,8711,B
CC,4898,-4898,CC
FC,4805,4805,FC
RB,3795,-3795,RB
KC,3487,3487,KC
SB,2216,2216,SB
LN,1516,1516,LN


In [15]:
import math as m
m.sqrt(52)*total_pnl_frame['portfolio'].mean()/total_pnl_frame['portfolio'].std()

# Rule 1 Sharp: 4.5



4.534496285051776

In [4]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display
rank_indicators_output = bs.rank_indicators(trade_data=big_data,
                                            indicator_list = ['Q', 'QF', 'z1', 'z2', 'z3', 'z4'],
                                            long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short',
                                            strategy_class='futures_butterfly')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_tickerhead_total = rank_indicators_output['indicator_ranking_tickerhead_total']
display(indicator_ranking_total)
display(indicator_ranking_tickerhead_total)

# Z1,QF combo seems to provide the best results


,indicator,ranking
22,"[z2, QF]",68
25,"[z2, z4]",67
34,"[z4, z2]",60
27,"[z3, QF]",58
17,"[z1, QF]",58
30,"[z3, z4]",56
35,"[z4, z3]",54
13,"[QF, z2]",54
20,"[z1, z4]",53
32,"[z4, QF]",51


,indicator,ranking
17,"[z1, QF]",919
22,"[z2, QF]",882
34,"[z4, z2]",874
27,"[z3, QF]",872
25,"[z2, z4]",865
32,"[z4, QF]",828
31,"[z4, Q]",828
6,"[Q, QF]",817
33,"[z4, z1]",816
35,"[z4, z3]",811


In [5]:
bs.get_indicator_rr_double_table(trade_data=big_data,indicator_list=['z1','QF'],
                                 strategy_class='futures_butterfly',
                                 long_pnl_field='hold_pnl5long',
                                            short_pnl_field='hold_pnl5short')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,-0.36,12,706.946080,0.264254
1,-0.36,48,299.715623,0.102080
2,-0.36,NaN,-105.066328,-0.034103
3,0.34,28,282.041004,0.108636
4,0.34,67,-120.849326,-0.043074
5,0.34,NaN,-308.224740,-0.097570
6,NaN,49,132.719829,0.040779
7,NaN,85,451.399935,0.153100
8,NaN,NaN,1078.828160,0.426178


In [5]:
big_data['tickerClass'].unique()

array(['Energy', 'Ag', 'Soft', 'STIR', 'Livestock'], dtype=object)

In [3]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

trade_data = big_data

pnl_field_list = ['hold_pnl5','hold_pnl10','hold_pnl20']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),                
                         ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                     
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

# waiting for a month seems to provide better results although this is more true for short then longs

,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.480690,1128.352616,7988736.5215
1,short2,0.449636,1238.656466,8842768.5095
2,long1,0.234923,589.985503,4359992.8640
3,long2,0.362463,1077.822369,7375538.4705


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.582468,1602.311568,11350775.1490
1,short2,0.656732,2136.795449,15260993.0985
2,long1,0.171598,557.994851,4127487.9115
3,long2,0.468563,1642.030770,11242984.6830


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.664447,2263.439003,16031938.4555
1,short2,0.742027,3070.654086,21936752.7935
2,long1,0.123692,582.482763,4316197.2765
3,long2,0.531095,2293.949362,15715847.0805


In [10]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

trade_data = big_data[big_data['tickerClass']=='Energy']

pnl_field_list = ['hold_pnl5_per_contract','hold_pnl10_per_contract','hold_pnl20_per_contract']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),                
                         ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                     
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

# waiting for a month seems to provide better results although this is more true for short then longs


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.553361,3.169000,13065.788470
1,short2,0.382752,2.989398,12008.413384
2,long1,0.093838,0.787559,3232.930763
3,long2,0.300599,2.791276,12033.192331


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.599038,4.329461,17880.673257
1,short2,0.664378,5.933171,23886.945048
2,long1,-0.005557,-0.070621,-289.970109
3,long2,0.479416,4.885367,21075.475002


,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.869328,7.460688,30812.642282
1,short2,1.325926,12.269818,49508.717079
2,long1,-0.129513,-2.652161,-10910.991382
3,long2,0.455441,6.112272,26423.352873


In [22]:
# some interesting results when you compare these iwth fixed holding periods
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

trade_data = big_data


pnl_field_list = ['path_pnl5_per_contract',
                  'path_pnl10_per_contract',
                  'path_pnl15_per_contract',
                  'path_pnl20_per_contract',
                  'path_pnl25_per_contract']

for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),
                          ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                                   
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)

,tier,reward_risk,mean_pnl,total_pnl
0,short1,3.460401,5.174432,36329.683649
1,short2,3.795920,9.489801,66893.606245
2,long1,0.259133,1.231704,9114.611967
3,long2,2.554748,9.449739,63747.939878


,tier,reward_risk,mean_pnl,total_pnl
0,short1,3.396150,6.080134,42664.300782
1,short2,3.215697,10.900496,76826.692738
2,long1,0.162136,1.134242,8393.390407
3,long2,1.813641,9.915478,66869.980800


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.546106,6.686132,46916.586174
1,short2,2.359706,11.699662,82447.519486
2,long1,0.142708,1.281695,9484.540121
3,long2,1.466741,10.345209,69747.400584


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.013489,7.031633,49305.810505
1,short2,2.159577,12.380557,87208.643814
2,long1,0.107109,1.093764,8091.668421
3,long2,1.189466,10.313393,69532.898312


,tier,reward_risk,mean_pnl,total_pnl
0,short1,1.710191,7.040760,49355.726502
1,short2,2.043907,12.774421,89957.470671
2,long1,0.089112,1.012456,7490.145944
3,long2,1.088333,10.527027,70962.688875


In [17]:
trade_data = big_data
trade_data = big_data[big_data['tickerClass']=='STIR']


pnl_field_list = ['hold_pnl20_per_contract',
                  'path_pnl5_per_contract',
                  'path_pnl10_per_contract',
                  'path_pnl25_per_contract']


for pnl_field in pnl_field_list:
    standard_short_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']>=0.6) & 
                                                     (trade_data['z1']<1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    standard_short_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']>=1.2) &
                                                      (trade_data['QF']>=85)][pnl_field].values)
    
    standard_long_stats1 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-0.6) & 
                                                     (trade_data['z1']>-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    standard_long_stats2 = bs.get_summary_stats(trade_data[(trade_data['z1']<=-1.2) &
                                                      (trade_data['QF']<=12)][pnl_field].values)
    
    summary_frame = pd.DataFrame.from_items([('tier', ['short1','short2','long1','long2']), 
                         ('reward_risk', [standard_short_stats1['reward_risk'],
                                         standard_short_stats2['reward_risk'],
                                         standard_long_stats1['reward_risk'],
                                         standard_long_stats2['reward_risk']]),
                          ('mean_pnl', [standard_short_stats1['mean_pnl'],
                                         standard_short_stats2['mean_pnl'],
                                         standard_long_stats1['mean_pnl'],
                                         standard_long_stats2['mean_pnl']]),                                   
                        ('total_pnl', [standard_short_stats1['total_pnl'],
                                         standard_short_stats2['total_pnl'],
                                         standard_long_stats1['total_pnl'],
                                         standard_long_stats2['total_pnl']])])
    display(summary_frame)



,tier,reward_risk,mean_pnl,total_pnl
0,short1,0.476096,1.799184,2164.417876
1,short2,0.531498,3.184719,4222.936916
2,long1,-0.341289,-2.566543,-2984.889521
3,long2,0.195428,2.368267,2609.830118


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.955783,1.602806,1931.381085
1,short2,1.350432,2.372027,3164.284573
2,long1,-0.236454,-0.640781,-744.587098
3,long2,1.052927,3.550205,3933.627580


,tier,reward_risk,mean_pnl,total_pnl
0,short1,2.334555,1.983269,2385.873075
1,short2,0.994290,2.576290,3434.195171
2,long1,-0.308698,-1.068425,-1241.509494
3,long2,0.500763,2.898462,3211.496213


,tier,reward_risk,mean_pnl,total_pnl
0,short1,1.021459,2.043505,2446.075574
1,short2,0.654687,2.807425,3731.068214
2,long1,-0.258387,-1.251199,-1452.642556
3,long2,0.371302,3.480956,3846.455978


In [21]:
trade_data = big_data
standard_short1_trades = trade_data[(trade_data['z1']>=0.6) & (trade_data['z1']<1.2) &(trade_data['QF']>=85)]

standard_short1_trades

import matplotlib.pyplot as plt
#trade_data=big_data
plt.scatter(standard_short1_trades['hold_pnl20'],standard_short1_trades['path_pnl10'])
plt.grid()
plt.show()

In [32]:
trade_data = big_data
standard_short1_trades = trade_data[(trade_data['z1']>=0.6) & (trade_data['z1']<1.2) &(trade_data['QF']>=85)]
standard_short1_trades['hold_success'] = (standard_short1_trades['hold_pnl20']>0)

standard_short1_trades['hold_success'].sum()
len(standard_short1_trades.index)

bad_hold_short_trades = standard_short1_trades[standard_short1_trades['hold_success']==False]
print(bad_hold_short_trades['path_pnl10'].mean())

print(bad_hold_short_trades['hold_pnl20'].mean())

4891.461852669592
7214.242659720706


In [44]:
trade_data = big_data
standard_short1_trades = trade_data[(trade_data['z1']>=0.6) & (trade_data['z1']<1.2) &(trade_data['QF']>=85)]


print(standard_short1_trades['path_pnl10'].quantile(0.2))
print(standard_short1_trades['hold_pnl20'].quantile(0.2))

-764.019999999
-3406.5


In [33]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.scatter(bad_hold_short_trades['hold_pnl20'],bad_hold_short_trades['path_pnl10'])
plt.grid()
plt.show()

In [19]:
import matplotlib.pyplot as plt
#trade_data=big_data
plt.plot(trade_data['second_spread_weight_1'])
plt.grid()
plt.show()

In [1]:
import get_price.presave_price as pp
pp.generate_and_update_futures_data_file_4tickerhead(ticker_head='CL')

In [12]:
wuhu = trade_data[trade_data['path_pnl25']<-100000]
wuhu.iloc[5]

#legit extremes: 20130123 HOJ2013,HOK2013,HOK2013
#20120620 SF2013,SK2013,SX2013
#20101117 CTK2011,CTN2011,CTZ2011
#20100117 CTH2011,CTN2011,CTH2012
#20080910 EDX2008,EDG2009, EDM2009

ticker1                     CLQ2008
ticker2                     CLU2008
ticker3                     CLX2008
tickerHead                       CL
tickerClass                  Energy
trDte1                          206
trDte2                          227
trDte3                          270
multiplier                     1000
agg                               1
cBack                           100
Q                                19
QF                               13
z1                            -0.25
z2                            -0.14
z3                            -0.26
z4                             0.35
r1                            96.89
r2                            96.17
RC                             0.02
seasonality                    0.02
yield1                            0
yield2                        0.094
bf_price                      -0.07
noise_ratio                   0.569
alpha1                       -0.026
alpha2                       -0.038
residual_std1               

In [ ]:
import get_price.get_futures_price as gfp
hov2007_data = gfp.get_futures_price_preloaded(ticker='HOV2007',settle_date_from_exclusive=20070103)
hox2007_data = gfp.get_futures_price_preloaded(ticker='HOX2007',settle_date_from_exclusive=20070103)
hoz2007_data = gfp.get_futures_price_preloaded(ticker='HOZ2007',settle_date_from_exclusive=20070103)

In [ ]:

import matplotlib.pyplot as plt
plt.plot(wuhu['price_ratio'])
plt.grid()
plt.show()


In [ ]:
wuhu['price_ratio']

In [ ]:
import opportunity_constructs.futures_butterfly as opfb
import backtesting.backtest_futures_with_paths as bfp
import pandas as pd
import numpy as np
import backtesting.utilities as bu
import ta.strategy as ts
import os.path
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
futures_data_dictionary = {x: gfp.get_futures_price_preloaded(ticker_head=x) for x in cmi.futures_butterfly_strategy_tickerhead_list}
wuhu = bfp.get_backtest_summary_4_date(report_date=20070314,futures_data_dictionary=futures_data_dictionary)
kuzu = wuhu[wuhu['ticker1']=='CLM2010']
kuzu.loc[61]

In [ ]:
import opportunity_constructs.futures_butterfly as opfb
import backtesting.backtest_futures_with_paths as bfp
import pandas as pd
import numpy as np
import backtesting.utilities as bu
import ta.strategy as ts
import os.path
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
futures_data_dictionary = {x: gfp.get_futures_price_preloaded(ticker_head=x) for x in cmi.futures_butterfly_strategy_tickerhead_list}
wuhu = bfp.get_backtest_summary_4_date(report_date=20070314,futures_data_dictionary=futures_data_dictionary)
wuhu

In [ ]:
# R1 seems to be a good thing to look at in general 

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r1']>50)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r1']>50)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# R2 seems to be a good thing to look at in general 

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r2']>50)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r2']>50)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]

In [ ]:
# Recent Vol Ratio Does Seem to Have Certain Implications For Sizing

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['recent_vol_ratio']>1)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['recent_vol_ratio']>1)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
                'downside20': [standard_short_stats1['downside20'],
                              standard_short_stats2['downside20'],
                              standard_long_stats1['downside20'],
                              standard_long_stats2['downside20']],                  
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','total_pnl']]

In [ ]:
# Recent Vol Ratio Does Seem to Have Certain Implications For Sizing

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['recent_5day_pnl']<2*big_data['upside'])]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      ((big_data['recent_5day_pnl']>2*big_data['downside']))
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
                'downside20': [standard_short_stats1['downside20'],
                              standard_short_stats2['downside20'],
                              standard_long_stats1['downside20'],
                              standard_long_stats2['downside20']],                  
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','total_pnl']]

In [ ]:
standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)



standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r2']>50)&
                                                      (big_data['z2']>1.5)
                                                     ]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r2']>50) &
                                                      (big_data['z2']<-1.5)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# Ticker Head Based Analysis

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
import pandas as pd
import backtesting.backtest_stats as bs

ticker_head_summary_dataframe = pd.DataFrame()
ticker_head_summary_dataframe['ticker_head'] = ticker_head_list
ticker_head_summary_dataframe['short_rr'] = float('NaN')
ticker_head_summary_dataframe['long_rr'] = float('NaN')
ticker_head_summary_dataframe['short_sum'] = float('NaN')
ticker_head_summary_dataframe['long_sum'] = float('NaN')
ticker_head_summary_dataframe['short_r1'] = float('NaN')
ticker_head_summary_dataframe['short_z1'] = float('NaN')
ticker_head_summary_dataframe['long_r1'] = float('NaN')
ticker_head_summary_dataframe['long_z1'] = float('NaN')


for i in range(len(ticker_head_list)):
    
    selected_short_trades = big_data[(big_data['Q']>85) & (big_data['QF']>60) &(big_data['tickerHead']==ticker_head_list[i])]
    selected_long_trades = big_data[(big_data['Q']<15) & (big_data['QF']<40) &(big_data['tickerHead']==ticker_head_list[i])]
    
    
    standard_short_stats = bs.get_summary_stats(selected_short_trades['pnl_final'].values)
    standard_long_stats = bs.get_summary_stats(selected_long_trades['pnl_final'].values)
    
    
    
    ticker_head_summary_dataframe['short_rr'][i] = standard_short_stats['reward_risk']
    ticker_head_summary_dataframe['long_rr'][i] = standard_long_stats['reward_risk']
    
    ticker_head_summary_dataframe['short_sum'][i] = standard_short_stats['total_pnl']
    ticker_head_summary_dataframe['long_sum'][i] = standard_long_stats['total_pnl']
    
    ticker_head_summary_dataframe['short_r1'][i] = selected_short_trades['r1'].mean()
    ticker_head_summary_dataframe['short_z1'][i] = selected_short_trades['z1'].mean()
    
    ticker_head_summary_dataframe['long_r1'][i] = selected_long_trades['r1'].mean()
    ticker_head_summary_dataframe['long_z1'][i] = selected_long_trades['z1'].mean()
    
    
ticker_head_summary_dataframe

In [ ]:
# Ticker Head Based Analysis

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
import pandas as pd
import backtesting.backtest_stats as bs

ticker_head_summary_dataframe = pd.DataFrame()
ticker_head_summary_dataframe['ticker_head'] = ticker_head_list
ticker_head_summary_dataframe['short_rr'] = float('NaN')
ticker_head_summary_dataframe['long_rr'] = float('NaN')
ticker_head_summary_dataframe['short_sum'] = float('NaN')
ticker_head_summary_dataframe['long_sum'] = float('NaN')
ticker_head_summary_dataframe['short_r1'] = float('NaN')
ticker_head_summary_dataframe['short_z1'] = float('NaN')
ticker_head_summary_dataframe['long_r1'] = float('NaN')
ticker_head_summary_dataframe['long_z1'] = float('NaN')


for i in range(len(ticker_head_list)):
    
    selected_short_trades = big_data[(big_data['Q']>85) & 
                                     (big_data['QF']>60) &
                                     (big_data['r2']>50)&
                                     (big_data['z2']>1.5) &
                                     (big_data['tickerHead']==ticker_head_list[i])]
    selected_long_trades = big_data[(big_data['Q']<15) & 
                                    (big_data['QF']<40) &
                                     (big_data['r2']>50) &
                                     (big_data['z2']<-1.5) &
                                    (big_data['tickerHead']==ticker_head_list[i])]
    
    
    standard_short_stats = bs.get_summary_stats(selected_short_trades['pnl_2'].values)
    standard_long_stats = bs.get_summary_stats(selected_long_trades['pnl_2'].values)
    
    
    
    ticker_head_summary_dataframe['short_rr'][i] = standard_short_stats['reward_risk']
    ticker_head_summary_dataframe['long_rr'][i] = standard_long_stats['reward_risk']
    
    ticker_head_summary_dataframe['short_sum'][i] = standard_short_stats['total_pnl']
    ticker_head_summary_dataframe['long_sum'][i] = standard_long_stats['total_pnl']
    
    ticker_head_summary_dataframe['short_r1'][i] = selected_short_trades['r1'].mean()
    ticker_head_summary_dataframe['short_z1'][i] = selected_short_trades['z1'].mean()
    
    ticker_head_summary_dataframe['long_r1'][i] = selected_long_trades['r1'].mean()
    ticker_head_summary_dataframe['long_z1'][i] = selected_long_trades['z1'].mean()
    
    
ticker_head_summary_dataframe

In [ ]:
# Measuring Risk

import matplotlib.pyplot as plt
plt.scatter(big_data['recent_vol_ratio'],big_data['pnl_short5'])
plt.grid()
plt.show()





In [ ]:
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import backtesting.backtest_futures_butterfly as bfb
import pandas as pd
import shared.directory_names as dn

#portfolio_output2 = bfb.construct_futures_butterfly_portfolio(date_list = selected_dates)

portfolio_output1 = pd.read_pickle(dn.backtest_results_folder + '/futures_butterfly/portfolio.pkl')


import matplotlib.pyplot as plt
plt.plot(range(len(portfolio_output2.index)),portfolio_output2['portfolio'].cumsum(),
        range(len(portfolio_output2.index)),portfolio_output1['portfolio'].cumsum())
plt.grid()
plt.show()

In [ ]:
import backtesting.backtest_futures_butterfly as bfb
#wuhu = bfb.construct_futures_butterfly_portfolio(rule_no=2,date_list=selected_dates)
wuhu.loc[wuhu['pnl_short1'] < -1000,'pnl_final']=wuhu.loc[wuhu['pnl_short1'] < -1000,'pnl_short2']
wuhu
wuhu.loc[wuhu['pnl_short1'] < -1000]['pnl_short2']
wuhu.loc[wuhu['pnl_short1'] < -1000]['pnl_final']

In [ ]:

import backtesting.backtest_futures_butterfly as bfb
import contract_utilities.contract_meta_info as cmi
import numpy as np
import pandas as pd
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list



contributors_output1 = bfb.analyze_portfolio_contributors(date_list=selected_dates,date_from = 20000101,date_to = 20160101,rule_no=1)
contributors_output2 = bfb.analyze_portfolio_contributors(date_list=selected_dates,date_from = 20000101,date_to = 20160101,rule_no=2)






In [ ]:
import matplotlib.pyplot as plt
total_pnl1 = contributors_output1['total_pnl_frame']
total_pnl2 = contributors_output2['total_pnl_frame']
plt.plot(range(len(total_pnl1.index)),total_pnl1['portfolio'].cumsum(),
         range(len(total_pnl1.index)),total_pnl2['portfolio'].cumsum())
plt.legend(['no stop','stop'],frameon=False)
plt.grid()
plt.show()

In [ ]:
total_pnl1[total_pnl1['portfolio']<-200000]

sheet = backtest_output[58]

ho_sheet = sheet[sheet['tickerHead']=='HO']
ho_sheet.loc[(ho_sheet['Q']<=15)&(ho_sheet['QF']<=40)]


In [ ]:
import get_price.get_futures_price as gfp
futures_data = gfp.get_futures_price_4ticker(ticker='HOQ2007')

import matplotlib.pyplot as plt
plt.plot(futures_data['close_price'])
plt.grid()
plt.show()



In [ ]:
import backtesting.backtest_stats as bs
portfolio_stats1 = bs.get_summary_stats(total_pnl1['portfolio'])
portfolio_stats2 = bs.get_summary_stats(total_pnl2['portfolio'])

summary_table = pd.DataFrame({'class': ['Portfolio1', 'Portfolio2'],
              'reward_risk': [portfolio_stats1['reward_risk'],
                              portfolio_stats2['reward_risk']],
              'mean_pnl': [portfolio_stats1['mean_pnl'],
                           portfolio_stats2['mean_pnl']],
                'downside20': [portfolio_stats1['downside20'],
                              portfolio_stats2['downside20']],   
                 'downside5': [portfolio_stats1['downside5'],
                              portfolio_stats2['downside5']],              
              'total_pnl': [portfolio_stats1['total_pnl'],
                              portfolio_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','downside5','total_pnl']]



In [ ]:
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
import shared.calendar_utilities as cu
import opportunity_constructs.utilities as opUtil
import contract_utilities.expiration as exp

report_date=20120711
post_report_date = exp.doubledate_shift_bus_days(double_date=report_date,shift_in_days=-20)

futures_data_dictionary = {x: gfp.get_futures_price_preloaded(ticker_head=x) for x in cmi.futures_butterfly_strategy_tickerhead_list}


butterflies = bfb.backtest_futures_butterfly_4date(report_date=report_date,futures_data_dictionary=futures_data_dictionary )

ng_butterflies = butterflies[butterflies['tickerHead']=='S']
ng_short = ng_butterflies[(ng_butterflies['Q']<15) & (ng_butterflies['QF'] < 40)]       

ng_short.columns

ng_short[['ticker1','ticker2','ticker3','r1','r2','z1','z2','Q','QF','pnl_2']]





In [ ]:
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import signals.futures_signals as fs
import shared.statistics as stats
strategy_out = fs.get_futures_butterfly_signals({'ticker_list': ['NGV2016', 'NGX2016', 'NGF2017'],'date_to': 20151104})
strategy_out['noise_ratio']

In [ ]:
import futures_charts.futures_curve as fc
#fc.get_futures_curve_chart_4date(ticker_head='HO',settle_date=20070214)
aligned_output = fc.get_butterfly_panel_plot(report_date=20070214,id=124 )
#fc.get_butterfly_scatter_plot(report_date=20070214,id=124)
#wuhu['c1']['close_price'][0]-2*wuhu['c2']['close_price'][0]+wuhu['c3']['close_price'][0]

In [ ]:

import matplotlib.pyplot as plt
plt.plot(aligned_data['settle_date'],
         aligned_data['c1']['close_price'])
plt.grid()
plt.show()

In [ ]:
# Basic Stats:

import math as m
import numpy as np

total_pnl = [x['total_pnl'] for x in backtest_output]
print('daily sd :' + str(round(np.std(total_pnl)/m.sqrt(5))))
print('daily mean :' + str(round(np.mean(total_pnl)/5)))
print('annualized sharp :' + str(m.sqrt(52)*np.mean(total_pnl)/np.std(total_pnl)))




In [ ]:
portfolio_pnl = [x['total_pnl'] for x in backtest_output]


import matplotlib.pyplot as plt
plt.plot(np.cumsum(portfolio_pnl))
plt.grid()
plt.show()

In [ ]:

# Tickerhead Based Results:

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
pnl_ticker_head_dictionary = dict()

for ticker_head in ticker_head_list:
    
    pnl_path_per_ticker_head = []
    
    for backtest_result in backtest_output:
        ticker_head_result = backtest_result['pnl_per_tickerhead'][backtest_result['pnl_per_tickerhead']['tickerHead']==ticker_head]
        pnl_path_per_ticker_head.append(ticker_head_result['mean_long'].sum()+ticker_head_result['mean_short'].sum())
    
    pnl_ticker_head_dictionary[ticker_head] = pnl_path_per_ticker_head
  


In [ ]:
# Tickerhead Based Stats

import shared.statistics as stats
import pandas as pd
import numpy as np

ticker_head_stats = pd.DataFrame({'ticker_head': ticker_head_list})

ticker_head_stats['long_mean'] = float('NaN')
ticker_head_stats['long_rr'] = float('NaN')
ticker_head_stats['short_mean'] = float('NaN')
ticker_head_stats['short_rr'] = float('NaN')


short_trades_list = [x['short_trades'] for x in backtest_output]
long_trades_list = [x['long_trades'] for x in backtest_output]

short_trades_frame  = pd.concat(short_trades_list)
long_trades_frame  = pd.concat(long_trades_list)

long_trades_frame

for i in range(len(ticker_head_list)):
    long_pnl_tickerhead = long_trades_frame['pnl'][long_trades_frame['tickerHead'] == ticker_head_list[i]]
    short_pnl_tickerhead = short_trades_frame['pnl'][short_trades_frame['tickerHead'] == ticker_head_list[i]]
    
    if len(long_pnl_tickerhead) > 0:
        ticker_head_stats['long_mean'][i] =np.nanmean(long_pnl_tickerhead)
        long_downside = stats.get_number_from_quantile(y=long_pnl_tickerhead.values,quantile_list=[20],clean_num_obs=10)
        ticker_head_stats['long_rr'][i] = ticker_head_stats['long_mean'][i]/abs(long_downside[0])
        
        
        ticker_head_stats['short_mean'][i] =np.nanmean(short_pnl_tickerhead)
        short_downside = stats.get_number_from_quantile(y=short_pnl_tickerhead.values,quantile_list=[20],clean_num_obs=10)
        ticker_head_stats['short_rr'][i] = ticker_head_stats['short_mean'][i]/abs(short_downside[0])
    
    
    
    
    


ticker_head_stats



In [ ]:
# Impact of Z1

selected_short = short_trades_frame[short_trades_frame['z1']>(short_trades_frame['z1'].mean())]
selected_short_mean = selected_short['pnl'].mean()
selected_short_downside = stats.get_number_from_quantile(y=selected_short['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected short mean: ' + str(selected_short_mean))
print('selected short downside: ' + str(selected_short_downside[0]))
print('selected short risk reward : ' + str(selected_short_mean/abs(selected_short_downside[0])))

selected_long = long_trades_frame[long_trades_frame['z1']>(long_trades_frame['z1'].mean())]
selected_long_mean = selected_long['pnl'].mean()
selected_long_downside = stats.get_number_from_quantile(y=selected_long['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected long mean: ' + str(selected_long_mean))
print('selected long downside: ' + str(selected_long_downside[0]))
print('selected long risk reward : ' + str(selected_long_mean/abs(selected_long_downside[0])))

# Results are Encouraging



In [ ]:
# Impact of R1

selected_short = short_trades_frame[short_trades_frame['r1']<(short_trades_frame['r1'].mean())]
selected_short_mean = selected_short['pnl'].mean()
selected_short_downside = stats.get_number_from_quantile(y=selected_short['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected short mean: ' + str(selected_short_mean))
print('selected short downside: ' + str(selected_short_downside[0]))
print('selected short risk reward : ' + str(selected_short_mean/abs(selected_short_downside[0])))

selected_long = long_trades_frame[long_trades_frame['r1']<(long_trades_frame['r1'].mean())]
selected_long_mean = selected_long['pnl'].mean()
selected_long_downside = stats.get_number_from_quantile(y=selected_long['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected long mean: ' + str(selected_long_mean))
print('selected long downside: ' + str(selected_long_downside[0]))
print('selected long risk reward : ' + str(selected_long_mean/abs(selected_long_downside[0])))

# Results are Encouraging

In [ ]:
short_trades_frame['z1'].mean()

In [ ]:
backtest_output[1]['pnl_per_tickerhead']

In [ ]:
butt = backtest_output[1]['butterflies']
butt['pnl'] = backtest_output[1]['butterfly_pnl']

ng_trades = butt[butt['tickerHead']=='ED']


short_trades = ng_trades[(ng_trades['Q']>85) & (ng_trades['QF']>60)]


short_trades[['ticker1','ticker2','ticker3','z1','z2','second_spread_weight_1','Q','QF','pnl']]


In [ ]:
short_trades.columns
